In [84]:
#Importing the dataset
import pandas as pd

df1=pd.read_csv('textdata.csv').fillna(' ')

In [85]:
df2=pd.read_csv('texttestdata.csv').fillna(' ')

In [86]:
df=pd.concat([df1,df2])

In [87]:
df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0.0,0.0,0.0,0.0,0.0,0.0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0.0,0.0,0.0,0.0,0.0,0.0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0.0,0.0,0.0,0.0,0.0,0.0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0.0,0.0,0.0,0.0,0.0,0.0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0.0,0.0,0.0,0.0,0.0,0.0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0.0,0.0,0.0,0.0,0.0,0.0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1.0,1.0,1.0,0.0,1.0,0.0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0.0,0.0,0.0,0.0,0.0,0.0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0.0,0.0,0.0,0.0,0.0,0.0
9,00040093b2687caa,alignment on this subject and which are contra...,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
df.shape

(312735, 8)

In [90]:
#Removing special characters from comment_text
import re

def clean_text(text):
    return re.sub(r'\W+',' ', text)

In [92]:
df['comment_text_clean']=df['comment_text']
df['comment_text_clean']=df['comment_text_clean'].apply(clean_text)

In [93]:
#Train data
df1['comment_text_clean']=df1['comment_text']
df1['comment_text_clean']=df1['comment_text_clean'].apply(clean_text)
#Test data
df2['comment_text_clean']=df2['comment_text']
df2['comment_text_clean']=df2['comment_text_clean'].apply(clean_text)

In [94]:
#FormattingText
def changecase(text):
    return text.lower()

In [95]:
df['comment_text_clean']=df['comment_text']
df['comment_text_clean']=df['comment_text_clean'].apply(changecase)

In [96]:
#Train data
df1['comment_text_clean']=df1['comment_text']
df1['comment_text_clean']=df1['comment_text_clean'].apply(changecase)
#Test data
df2['comment_text_clean']=df2['comment_text']
df2['comment_text_clean']=df2['comment_text_clean'].apply(changecase)

In [103]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer=CountVectorizer(stop_words='english', min_df=0.0001,analyzer='word',max_features=10000)
X_train=vectorizer.fit(df['comment_text_clean'])
X_train=vectorizer.transform(df1['comment_text_clean'])
X_test=vectorizer.transform(df2['comment_text_clean'])

In [99]:
#Check if the dataset is balanced -
len(df[df['identity_hate']==1])

1405

In [100]:
df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [101]:
target = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [104]:
#Building SVC model
from sklearn.svm import LinearSVC

svm_clf_m1=LinearSVC(random_state=42)
result=pd.DataFrame()

for labels in target:
    svm_clf_m1.fit(X_train,df1[labels])
    result[labels]=svm_clf_m1.predict(X_train)

result

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [105]:
len(result[result['toxic']==1])

12936

In [106]:
#Using TFIDF as the feature 
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english',min_df=0.0001,analyzer='word',max_features=10000)
X_train_2=tfidf_vectorizer.fit(df['comment_text_clean'])
X_train_2=tfidf_vectorizer.transform(df1['comment_text_clean'])
X_test_2=tfidf_vectorizer.transform(df2['comment_text_clean'])

In [108]:
result_1=pd.DataFrame()
svm_clf_m2=LinearSVC(random_state=42)

for labels in target:
    svm_clf_m2.fit(X_train_2,df1[labels])
    result_1[labels]=svm_clf_m2.predict(X_train_2)

result_1

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [109]:
len(result_1[result_1['toxic']==1])

12370

In [110]:
X_train_2

<159571x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 3530188 stored elements in Compressed Sparse Row format>

In [113]:
#Running predict function on both models
result_test_m1=pd.DataFrame()
result_test_m2=pd.DataFrame()

for labels in target:
    result_test_m1[labels]=svm_clf_m1.predict(X_test)
    result_test_m2[labels]=svm_clf_m2.predict(X_test)

print(len(result_test_m1[result_test_m1['toxic']==1]),len(result_test_m2[result_test_m2['toxic']==1]))

1777 16470
